In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))


## Why this kernel
   In this competition,I believe many of us use the leak to get a better score.
   
  Can get a reference from here:[https://www.kaggle.com/tezdhar/breaking-lb-fresh-start](https://www.kaggle.com/tezdhar/breaking-lb-fresh-start)
  
 It was originally from Giba's share.[post](https://www.kaggle.com/c/santander-value-prediction-challenge/discussion/61329)
 
 And, 'Jiazhen Xi' Extended it.[kernel](https://www.kaggle.com/johnfarrell/giba-s-property-extended-extended-result)
 
 But, all those work are base on Giba given some rows and columns. Actually,I'm not quite understand how to get them.maybe someone can teach me.
 
 so,I am wondering if there is any way to search for some leaks.This is the exploration of this kernel.

In [ ]:
train = pd.read_csv('../input/train.csv')

In [ ]:
# Horizontal search, to find more columns
# reference:
# https://www.kaggle.com/johnfarrell/giba-s-property-extended-extended-result
def bf_search_a(df_new, df_cand):
    cnt = 0
    head_curr = df_new.values[1:, 0]
    tail_curr = df_new.values[:-1, -1]
    while True:
        for c in df_cand.columns:
            if c in df_new:
                continue
            elif np.all(
                df_cand[c].iloc[:-1].values==head_curr
            ) and len(df_cand[c].unique())>1:
                df_new.insert(0, c, df_cand[c].values)
                head_curr = df_new.values[1:, 0]
#                 print(c, 'found head!', 'new shape', df_new.shape)
                cnt += 1
                break
            elif np.all(
                df_cand[c].iloc[1:].values==tail_curr
            ) and len(df_cand[c].unique())>1:
                df_new[c] = df_cand[c].values
                tail_curr = df_new.values[:-1, -1]
#                 print(c, 'found tail!', 'new shape', df_new.shape)
                cnt += 1
                break
            else:
                continue
        if cnt==0:
            break
        else:
            cnt = 0
            continue
    return df_new

In [ ]:
# vertically Search,to find more rows
def df_append(df, index, train, target='target', col='col', index_h={}):
    index_h.add(index)
    df = df.append(train[[target,col]].loc[index])
    last_value = df[target].tail(1).values[0]
    col_index = list(train[train[col]==last_value].index)
    uni_col_index = list(set(col_index) - index_h)

    if uni_col_index:
        if df.shape[0] < 2:
            df = df_append(df, min(uni_col_index), train, target, col, index_h)  
        for col_index_i in uni_col_index:
            uni_col_index.remove(col_index_i)
            df_c = df.copy()
            bf_search_a(df_c, train.iloc[df_c.index, 2:])
            if df_c.shape[1] >= 4:
                print(df_c.index, df_c.shape)
                df = df_append(df, col_index_i, train, target, col, index_h)
            else:
                df = df.drop(df.index[-1])
                df = df.append(train[[target,col]].loc[col_index_i])
    return df

In [ ]:
## run to search
# for col in [f for f in train.columns if f not in['ID','target']]:  # use this to search in all columns
for col in [f for f in train.columns if f not in['ID','target']][1:3]:
    print('#################### new col:     ',col)
#     for i in range(train.shape[0]):  # use this to search in all rows
    for i in range(50):
        ff = pd.DataFrame()
        index_h = set()
        df_append(ff, i, train, 'target', col, index_h)

## But......
but it seems to need to run a lot of time,In fact, I have not finished it.

it maybe helps,maybe not.